# End-to-end NLP: News Headline classifier

### Setup execution role and session

In [ ]:
import numpy as np
import pandas as pd

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting. If you don't specify a bucket, SageMaker SDK will create a default bucket following a pre-defined naming convention in the same region. 
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
%%time
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
print(role)
sess = sagemaker.Session()
#bucket = <bucket> # custom bucket name.
s3_bucket = sess.default_bucket()
s3_prefix = 'news'

### Download News Aggregator Dataset available at the public UCI dataset repository

We will download our dataset from the UCI Machine Learning Database public repository. The dataset is the News Aggregator Dataset and we will use the newsCorpora.csv file. This dataset contains a table of news headlines and their corresponding classes.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip

In [ ]:
!unzip NewsAggregatorDataset.zip

In [ ]:
!rm -rf __MACOSX/

In [ ]:
ls

### Let's visualize the dataset

We will load the newsCorpora.csv file to a Pandas dataframe for our data processing work.

In [ ]:
import pandas as pd
import mxnet
import re
import numpy as np
import os

In [ ]:
column_names = ["TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]
news_dataset = pd.read_csv('newsCorpora.csv', names=column_names, header=None, delimiter='\t')
news_dataset.head()

#### For this exercice we'll only use the title (Headline) of the news story and the category as our target variable

In [ ]:
df=news_dataset[['TITLE',"CATEGORY"]]

In [ ]:
from collections import Counter
Counter(df['CATEGORY'])

The dataset has four categories: Business (b), Science & Technology (t), Entertainment (e) and Health & Medicine (m).

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
leMapped=le.fit_transform(df["CATEGORY"].values)
list(le.classes_)

## Natural Language pre processing

We will do some basic processing of the text data to convert it into numerical form that the algorithm will be able to consume to create a model.
We will do typical pre processing for NLP workloads such as: dummy encoding the labels, tokenizing the documents and set fixed sequence lengths for input feature dimension, padding documents to have fixed length input vectors.

#### Dummy encode the labels

In [ ]:
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical
encoder = preprocessing.LabelEncoder()

docs = df["TITLE"].values

encoder.fit(df["CATEGORY"].values)
encoded_Y = encoder.transform(df["CATEGORY"].values)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [ ]:
list(encoder.classes_)

In [ ]:
encoded_Y

#### Tokenize documents and set fixed sequence lengths for input feature dimension.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print("Vocabulary size: " + str(vocab_size))
# pad documents to a max length of 4 words
max_length = 40
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print("Number of headlines: " + str(len(padded_docs)))

In [ ]:
docs[0]

In [ ]:
padded_docs[0]

### Import word embeddings

The vectors.txt file is the output of the blazingtext_word2vec_text8.ipynb notebook. This will have a list of vector representations for each word in our vocabulary.

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./vectors.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

##### Create embedding matrix

In [ ]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

### Import necessary keras libraries to build DL network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import KFold

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)


### Train test split to feed to model for evaluation and training.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, dummy_y, test_size=0.2, random_state=42)

### Build Deep Learning network

#### Build a keras Sequential feedforward model.
#### Embedding > Conv1D > MaxPooling > Flatten > Dropout > Dense > Dense (final Softmax layer)
#### First layer is an Embedding layer that will recieve as input our pre built word embeddings
#### We will use a 1D convolutional network to capture the sequential dimension of language (neigbouring words will be important when classifying context)
#### We will use dropout for regularization.
#### Finally we will use rmsporp optimization scheme.


In [ ]:
saveBestModelWeights = ModelCheckpoint("news_model_weights.h5",
                                       monitor='val_acc',
                                       verbose=1, 
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='auto',
                                       period=1)

    # define the model
model = Sequential()
model.add(Embedding(vocab_size, 100, 
                        weights=[embedding_matrix],
                        input_length=40, 
                        trainable=False, 
                        name="embed"))
model.add(Conv1D(filters=128, 
                     kernel_size=3, 
                     activation='relu',
                     name="conv_1"))
model.add(MaxPooling1D(pool_size=5,
                           name="maxpool_1"))
model.add(Flatten(name="flat_1"))
model.add(Dropout(0.3,
                     name="dropout_1"))
model.add(Dense(128, 
                    activation='relu',
                    name="dense_1"))
model.add(Dense(le.classes_.size,
                    activation='softmax',
                    name="out_1"))
    
    # compile the model
model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['acc'])
    

model.summary()


## Train model

In [ ]:
        
    # fit the model
model.fit(X_train,
              y_train,
              batch_size=16,
              epochs=5, # no benefit from additional epochs
              verbose=1,
              callbacks=[saveBestModelWeights])
    
scores = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))




# While we wait for this model to end training please continue to the next notebook (headline-classifier-mxnet.ipynb)

In [ ]:
example_doc=['Senate prepares to vote on dueling plans to end shutdown']
# integer encode the document
encoded_example = t.texts_to_sequences(example_doc)

# pad documents to a max length of 4 words
max_length = 40
padded_example = pad_sequences(encoded_example, maxlen=max_length, padding='post')

In [ ]:
model.predict(padded_example)